In [1]:

import os
import sys
import importlib
import copy
from collections import defaultdict
sys.path.insert(0, '/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc')

from tools_pattern import get_eucledean_dist

# script_n = os.path.basename(__file__).split('.')[0]
script_n = 'distribution_123share_bouton_210519_random_gen'

import my_plot
importlib.reload(my_plot)
from my_plot import MyPlotData, my_box_plot

def to_ng_coord(coord):
    return (
        int(coord[0]/4),
        int(coord[1]/4),
        int(coord[2]/40),
        )

import compress_pickle

# input_graph = compress_pickle.load('/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc/'\
#                                    'mf_grc_model/input_graph_201114_restricted_z.gz')
fname = ('/n/groups/htem/Segmentation/shared-nondev/cb2_segmentation/analysis_mf_grc/' \
#          'gen_db/mf_grc/input_graph_210519_all.gz')
         'gen_db/mf_grc/input_graph_210520_all_100_2.gz')
input_graph = compress_pickle.load(fname)

# z_min = 19800
# z_max = 29800
z_min = 19800
z_max = 29800
# GrCs are fully reconstructed and proofread from 90k to 150k
x_min = 105*1000*4
x_max = 135*1000*4
# radius = 200

n_randoms = 5
replication_hist2 = defaultdict(int)
grc_ids = set()
mf_ids = set()
replicated_2shares = defaultdict(int)

def get_prob(in_graph, unique_count=False, count_within_box=True, return_counted=False):
    n_common_pairs = 0
    processed = set()
    total_n_pairs = 0
    hist = defaultdict(int)
    n = 0
    counted_grcs = 0
    for grc_i_id in in_graph.grcs:
        n += 1
        grc_i = in_graph.grcs[grc_i_id]
        x, y, z = grc_i.soma_loc
        if count_within_box:
            if x < x_min or x > x_max:
                continue
            if z < z_min or z > z_max:
                continue
        counted_grcs += 1
        grc_ids.add(grc_i_id)
        rosettes_i = set([mf[1] for mf in grc_i.edges])
        for r in rosettes_i:
            mf_ids.add(r)
        for grc_j_id in in_graph.grcs:
            if grc_i_id == grc_j_id:
                continue
            if unique_count and (grc_i_id, grc_j_id) in processed:
                continue
            processed.add((grc_i_id, grc_j_id))
            processed.add((grc_j_id, grc_i_id))
            grc_j = in_graph.grcs[grc_j_id]
            x, y, z = grc_j.soma_loc
#             if count_within_box:
#                 if x < x_min or x > x_max:
#                     continue
#                 if z < z_min or z > z_max:
#                     continue
            common_rosettes = set([mf[1] for mf in grc_j.edges])
            common_rosettes = common_rosettes & rosettes_i
            hist[len(common_rosettes)] += 1
            if len(common_rosettes) == 2:
                replication_hist2[grc_i_id] += 1
                common_rosettes = tuple(sorted(list(common_rosettes)))
                replicated_2shares[common_rosettes] += 1
    for k in hist:
        # fix 0 datapoint plots
        if hist[k] == 0:
            hist[k] = 1
    if return_counted:
        return hist, counted_grcs
    else:
        return hist


In [7]:
n_random = 100
rounds = []
for n in range(n_random):
    print('', end='.')
    input_observed = input_graph
    input_observed.randomize_graph_by_grc2(
        constant_dendrite_length=10000,
        mf_dist_margin=10000,
        seed=n,
    )

    replication_hist2 = defaultdict(int)
    hist_data, n_grcs = get_prob(input_observed, count_within_box=True, return_counted=True)
#     print(hist_data)

    replication_hist2_list = []
    for grc in grc_ids:
        if grc in replication_hist2:
            replication_hist2_list.append((grc, replication_hist2[grc]))
        else:
            replication_hist2_list.append((grc, 0))
    replication_hist2_list_sorted = sorted(replication_hist2_list, key=lambda x: x[1])
    
    l = []
    for mf_id in replication_hist2_list_sorted:
        mf_id, size = mf_id
        l.append(size)
    rounds.append(l)

#     mpd = MyPlotData()
#     mpd_count = MyPlotData()
#     i = 0
#     for grc_id, count in replication_hist2_list_sorted:
#         mpd_count.add_data_point(
#             count=count,
#             grc_id=grc_id,
#             i=i,
#             model='Observed',
#         )
#         i += 1


....................................................................................................

In [8]:
import compress_pickle
fname = f'{script_n}_circle_constant10_2_{n_random}.gz'
print(fname)
compress_pickle.dump(rounds, fname)

distribution_123share_bouton_210519_random_gen_circle_constant10_2_100.gz
